# 🔁 Continual Training Lab: Delta Dataset Fine-Tuning
In this notebook, you will:
- Load hallucination failure cases
- Add corrected outputs
- Save a delta dataset
- Retrain the model on just the patch
- Prompt for re-evaluation

## 📥 Load Previous Evaluation Failures

In [ ]:
import json
with open('logs/hallucination_eval_results.json') as f:
    results = json.load(f)
failures = [r for r in results if r['hallucinated']]
print(f"Loaded {len(failures)} hallucinated examples.")
failures[:1]

## ✍️ Add Corrected Outputs

In [ ]:
# Replace <insert correct answer here> manually or use GPT assistance
corrected = []
for f in failures:
    corrected.append({
        "instruction": f["prompt"],
        "output": "<insert correct answer here>"
    })

with open("data/hallucination_corrections.jsonl", "w") as out:
    for ex in corrected:
        out.write(json.dumps(ex) + "\n")

## 🔁 Fine-Tune Using Delta Set

In [ ]:
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/zephyr-1.3b-bnb-4bit',
    max_seq_length=2048,
    load_in_4bit=True
)
FastLanguageModel.for_inference(model)

delta = load_dataset("json", data_files="data/hallucination_corrections.jsonl")['train']
delta = delta.map(lambda ex: {'prompt': ex['instruction'], 'completion': ex['output']})

args = TrainingArguments(
    output_dir='./checkpoints/model_v1.1',
    per_device_train_batch_size=2,
    num_train_epochs=2,
    fp16=True,
    save_strategy='epoch'
)

model.train_model(dataset=delta, args=args)

## ✅ Evaluate After Retraining

In [ ]:
print("Model updated. Please re-run hallucination_eval.ipynb with ./checkpoints/model_v1.1")